In [1]:
import os
os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

In [2]:
from banzai.calibrations import make_master_calibrations
import requests
from banzai_nres import settings
from banzai import dbs
from banzai.utils.stage_utils import run_pipeline_stages
import logging
from banzai.logs import set_log_level
from glob import glob

import pkg_resources

Setup logging and some settings show we know where the DB should be.
------

In [3]:
set_log_level('DEBUG')
logger = logging.getLogger('banzai')

In [4]:
from banzai import main

2020-05-21 10:54:10.302     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "lscnrs01-fl09-20180313-0005-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:10.525     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0005-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "master_calibration": "bpm-lsc-nres01-fl09-20180215.fits.fz", "processName": "MainProcess"}
2020-05-21 10:54:10.563     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "lscnrs01-fl09-20180313-0005-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:10.619 

2020-05-21 10:54:21.443     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "lscnrs01-fl09-20180313-0015-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:21.472     INFO:          stages: Running banzai.stages.GainNormalizer | {"filename": "lscnrs01-fl09-20180313-0015-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:21.472     INFO:            gain: Multiplying by gain | {"filename": "lscnrs01-fl09-20180313-0015-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:21.506     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "l

2020-05-21 10:54:32.333     INFO:            gain: Multiplying by gain | {"filename": "lscnrs01-fl09-20180313-0011-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:32.368     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "lscnrs01-fl09-20180313-0011-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:32.369     INFO:            trim: Trimming image | {"filename": "lscnrs01-fl09-20180313-0011-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:32.625     INFO:          stages: Running banzai.stages.PoissonInitializer | {"filename": "lscnrs01-fl09-20180313-

2020-05-21 10:54:43.647     INFO:          stages: Running banzai.stages.PoissonInitializer | {"filename": "lscnrs01-fl09-20180313-0001-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:43.793     INFO:          stages: Running banzai.stages.BiasSubtractor | {"filename": "lscnrs01-fl09-20180313-0001-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:44.458     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0001-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-bias-bin1x1.fits.fz", "processName": "MainProcess"}
2020-05-21 10:54:44.63

2020-05-21 10:54:55.085     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-bias-bin1x1.fits.fz", "processName": "MainProcess"}
2020-05-21 10:54:55.252     INFO:          stages: Running banzai.stages.DarkSubtractor | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:54:55.940     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-dark-bin1x1.fits.fz

...

2020-05-21 10:57:00.415     INFO:          traces: Fitting smooth spline to order 14 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:00.742     INFO:          traces: Fitting smooth spline to order 15 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:01.073     INFO:          traces: Fitting smooth spline to order 16 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:01.401     INFO:          traces: Fitting smooth spli

2020-05-21 10:57:09.252     INFO:          traces: Fitting smooth spline to order 41 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:09.582     INFO:          traces: Fitting smooth spline to order 42 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:09.911     INFO:          traces: Fitting smooth spline to order 43 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:10.262     INFO:          traces: Fitting smooth spli

2020-05-21 10:57:18.223     INFO:          traces: Fitting smooth spline to order 68 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:18.549     INFO:          traces: Fitting smooth spline to order 69 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:18.870     INFO:          traces: Fitting smooth spline to order 70 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:19.194     INFO:          traces: Fitting smooth spli

2020-05-21 10:57:26.969     INFO:          traces: Fitting smooth spline to order 95 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:27.290     INFO:          traces: Fitting smooth spline to order 96 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:27.613     INFO:          traces: Fitting smooth spline to order 97 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:27.939     INFO:          traces: Fitting smooth spli

2020-05-21 10:57:35.867     INFO:          traces: Fitting smooth spline to order 122 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:36.189     INFO:          traces: Fitting smooth spline to order 123 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:36.512     INFO:          traces: Fitting smooth spline to order 124 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-05-21 10:57:36.855     INFO:          traces: Fitting smooth s

In [5]:
os.environ['DB_ADDRESS'] = 'sqlite:///test_data/test.db'

settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']

Download some test data from the archive
----------------------------------------

In [6]:
# make directories for the test dataset.
raw_data_dir = 'test_data/lsc/nres01/20180313/raw'
os.makedirs(raw_data_dir, exist_ok=True)
bpm_dir = 'test_data/lsc/nres01/bpm/'
os.makedirs(bpm_dir, exist_ok=True)

In [7]:
# choose which files we will download for our test dataset.
bpm_filename = 'bpm-lsc-nres01-fl09-20180215.fits.fz'
test_filenames = ['lscnrs01-fl09-20180313-0001-w00.fits.fz', 'lscnrs01-fl09-20180313-0002-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0003-w00.fits.fz', 'lscnrs01-fl09-20180313-0004-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0005-w00.fits.fz', 'lscnrs01-fl09-20180313-0006-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0007-a00.fits.fz', 'lscnrs01-fl09-20180313-0008-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0009-w00.fits.fz', 'lscnrs01-fl09-20180313-0010-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0011-w00.fits.fz', 'lscnrs01-fl09-20180313-0012-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0013-w00.fits.fz', 'lscnrs01-fl09-20180313-0014-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0015-w00.fits.fz', 'lscnrs01-fl09-20180313-0016-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0017-w00.fits.fz', 'lscnrs01-fl09-20180313-0018-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0019-a00.fits.fz', 'lscnrs01-fl09-20180313-0020-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0021-a00.fits.fz', 'lscnrs01-fl09-20180313-0022-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0023-a00.fits.fz', 'lscnrs01-fl09-20180313-0042-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0043-b00.fits.fz', 'lscnrs01-fl09-20180313-0044-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0045-b00.fits.fz', 'lscnrs01-fl09-20180313-0046-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0047-d00.fits.fz', 'lscnrs01-fl09-20180313-0048-d00.fits.fz',
                  'lscnrs01-fl09-20180313-0049-d00.fits.fz', 'lscnrs01-fl09-20180313-0028-e00.fits.fz',
                  'lscnrs01-fl09-20180313-0029-e00.fits.fz', 'lscnrs01-fl09-20180313-0030-e00.fits.fz',
                  'lscnrs01-fl09-20180313-0031-e00.fits.fz']

bpm_frame_id = '22529799'
test_frame_ids = ['8148793', '8148822', '8148805', '8148824', '8148826', '8148876', '8148898',
                  '8148932', '8148960', '8148978', '8149024', '8149104', '8149068', '8149090',
                  '8149128', '8149173', '8149217', '8151252', '8149314', '8149388', '8149469',
                  '8149547', '8149570', '8156341', '8156360', '8156366', '8156376', '8156385',
                  '8156433', '8156500', '8156560', '8151997', '8152050', '8152110', '8152153']

In [ ]:
# DO NOT RUN ME IF YOU HAVE ALREADY REDUCED THE DATA BEFORE
# Copy the data into the correct directory
for test_filename, frame_id in zip(test_filenames, test_frame_ids):
    archive_url = f'https://archive-api.lco.global/frames/{frame_id}'
    frame_info = requests.get(archive_url).json()
    with open(os.path.join(raw_data_dir, test_filename), 'wb') as f:
        f.write(requests.get(frame_info['url']).content)
        
archive_url = f'https://archive-api.lco.global/frames/{bpm_frame_id}'
frame_info = requests.get(archive_url).json()
with open(os.path.join(bpm_dir, bpm_filename), 'wb') as f:
    f.write(requests.get(frame_info['url']).content)

Initialize the db and context object for the rest of the pipeline
-----------------------------------------------------------------

In [8]:
# set up the context object.
import banzai
context = banzai.main.parse_args(settings, parse_system_args=False)

In [9]:
# DO NOT RUN ME IF YOU HAVE ALREADY REDUCED THE DATA BEFORE
os.system(f'banzai_create_db --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site lsc --camera fl09 --name nres01 --camera-type 1m0-NRES-SciCam --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site elp --camera fl17 --name nres02 --camera-type 1m0-NRES-SciCam --db-address={os.environ["DB_ADDRESS"]}')
for bpm_filename in glob('test_data/*/nres??/bpm/*bpm*'):
    logger.info(f'adding bpm {bpm_filename} to the database')
    os.system(f'banzai_nres_add_bpm --filename {bpm_filename} --db-address={os.environ["DB_ADDRESS"]}')

Process and Stack Bias Files
----------------------------

In [11]:
bias_files = glob(os.path.join('test_data/*/nres??/*/raw/*b00*'))
for bias_file in bias_files: 
    run_pipeline_stages([{'path': bias_file}], context) 

In [12]:
def mark_frames_as_good(filenames):
    for filename in glob(f'test_data/*/nres??/*/processed/{filenames}'):
        dbs.mark_frame(os.path.basename(filename), "good", db_address=os.environ['DB_ADDRESS'])

In [13]:
mark_frames_as_good('*b91*')

In [9]:
instrument = dbs.get_instruments_at_site('lsc', settings.db_address)[0]

In [15]:
make_master_calibrations(instrument, 'BIAS', '2017-01-01', '2019-01-01', context) 

Process and Stack Dark Files
----------------------------

In [16]:
dark_files = glob('test_data/*/nres??/*/raw/*d00*')
for dark_file in dark_files: 
    run_pipeline_stages([{'path': dark_file}], context)

In [17]:
mark_frames_as_good('*d91*')

In [18]:
make_master_calibrations(instrument, 'DARK', '2017-01-01', '2019-01-01', context) 

Process and stack lamp flats. This is where we also solve for the traces and profile.
----------

In [10]:
flat_files = glob('test_data/*/nres??/*/raw/*w00*')
for flat_file in flat_files: 
    run_pipeline_stages([{'path': flat_file}], context)

In [11]:
make_master_calibrations(instrument, 'LAMPFLAT', '2017-01-01', '2019-01-01', context)

Process and stack arc lamps. Find the wavelength solution
--------

In [12]:
arc_files = glob('test_data/*/nres??/*/raw/*a00*')
for arc_file in arc_files: 
    run_pipeline_stages([{'path': arc_file}], context)

In [13]:
make_master_calibrations(instrument, 'DOUBLE', '2017-01-01', '2019-01-01', context) 

Process the science spectra
------

In [ ]:
science_files = glob('test_data/*/nres??/*/raw/*e00*')
for science_file in science_files: 
    run_pipeline_stages([{'path': science_file}], context)